# Initialization

In [ ]:
# Parameters. 
# Times Square replaces this cell with the user's parameters.

# The run-time defaults for all of these parameters are in NightLog.yaml
# Under Times Square, the run-time defaults always override values given here.
# Values here are used for local tests.

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
day_obs = '2024-09-04' # Value to use for local testing (Summit)
#!day_obs = 'TODAY' # TODO Change to 'TODAY' to test with default before push  

# Total number of days of data to display (ending on day_obs)
number_of_days = '12'  # TODO Change to '1' to test with default before push  

In [ ]:
# Only use packages available in the Rubin Science Platform
import requests
from collections import defaultdict
import pandas as pd
from pprint import pp
from urllib.parse import urlencode
from IPython.display import display, Markdown, display_markdown
from matplotlib import pyplot as plt
import os
from datetime import datetime, date, timedelta
#! from rubin_scheduler.site_models import Almanac

In [ ]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
limit = 50  # YAGNI: Auto get more if this isn't enough to get all requested DAYS

match day_obs.lower():
    case 'today':
        date = datetime.now().date()
    case 'yesterday':
        date = datetime.now().date()-timedelta(days=1)
    case _:
        date = datetime.strptime(day_obs, '%Y-%m-%d').date()
# date:  is EXLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
min_day_obs = (date - timedelta(days=days-1)).strftime('%Y%m%d') # Inclusive
max_day_obs = (date + timedelta(days=1)).strftime('%Y%m%d') # prep for Exclusive

response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds
timeout = (float(response_timeout), float(read_timeout))

In [ ]:
# Set default env to "usdf" and try before PUSH to repo.
summit = 'https://summit-lsp.lsst.codes'
usdf = 'https://usdf-rsp-dev.slac.stanford.edu'
tucson = 'https://tucson-teststand.lsst.codes'

# The default provided here is for local testing.
# Under Times Square it is ignored.
server = os.environ.get('EXTERNAL_INSTANCE_URL', summit) # TODO try with "usdf" before push (else "summit")

In [ ]:
# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get('EXTERNAL_INSTANCE_URL'):
    print('Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch....')
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype >/dev/null
import lsst.ts.logging_and_reporting.source_adapters as sad
import lsst.ts.logging_and_reporting.almanac as alm
from lsst.ts.logging_and_reporting.reports import md,mdlist,dict_to_md

In [ ]:
try:
    import lsst.ts.logging_and_reporting.version
    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = 'LIVE'

try:
    from lsst_efd_client import EfdClient
    enable_efd = True
except:
    enable_efd = False



# Overview

In [ ]:
md(f'''
Report for **{date}** covering the previous **{days}** observing night(s).
- Run on logs from **{server}/**
- Using *Prototype* Logging and Reporting Version: **{lrversion}**
- {enable_efd=}
''')

# Almanac

In [ ]:
a = alm.Almanac()
pd.DataFrame(a.as_dict).T   # TODO get rid of bogus header

# Nightly Report

In [ ]:
nr_adapter = sad.NightReportAdapter(server_url=server)
nr_url = nr_adapter.source_url
try:
    nr_recs,nr_url = nr_adapter.get_reports(
                                        limit=limit,
                                        min_day_obs=min_day_obs,
                                        max_day_obs=max_day_obs,
                                        )
except Exception as err:
    nr_recs = []
    msg = f'ERROR getting records from {nr_url=}: {err=}'
    raise Exception(msg)

In [ ]:
print(f'Retrieved {len(nr_recs)} records from {nr_url}')

In [ ]:
front = 'https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/'
tickets = nr_adapter.nightly_tickets(nr_recs)

if tickets:
    mdstr = '## Nightly Jira BLOCKs'
    for day, url_list in tickets.items():
        mdstr += f'\n- {day}'
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    md(f'No jira BLOCK tickets found.', color='lightblue')
    md(f'Used: [API Data]({nr_url})')
    

# Exposure Log

In [ ]:
exposure_adapter = sad.ExposurelogAdapter(server_url=server)
exposure_url = exposure_adapter.source_url
try:
    exposure_recs,url = exposure_adapter.get_messages(
                                        limit=limit,
                                        min_day_obs=min_day_obs,
                                        max_day_obs=max_day_obs,
                                        )
except Exception as err:
    exposure_recs = []
    msg = f'ERROR getting records from {url=}: {err=}'
    raise Exception(msg)

In [ ]:
if exposure_recs:
    table = exposure_adapter.day_table(exposure_recs,'date_added', dayobs_field='day_obs')
    #print(table)
    mdlist(table)
else:
    md(f'No Exposure Log records found.', color='lightblue')
    md(f'Used [API Data]({exposure_url})')

In [ ]:
if usdf == os.environ.get('EXTERNAL_INSTANCE_URL'):
    md(f"**Warning:** The `/exposures/` endpoint is not yet functional on SERVER=usdf.", color='red')
gaps = exposure_adapter.get_observation_gaps(min_day_obs=min_day_obs,
                                             max_day_obs=max_day_obs
                                            )
if gaps:
    md(f'### Date vs Observation Gap (minutes) for all Instruments')
    for instrument, day_gaps in gaps.items():
        if len(day_gaps) == 0:
            md(f'**No day gaps found for *{instrument=!s}* **', color='lightblue')
        else:
            x,y = zip(*day_gaps.items())
            df = pd.DataFrame(dict(day=x,minutes=y))
            df.plot.bar(x='day', y='minutes', title=f'{instrument=!s}')
else:
    md(f'No Observation Gaps found in exposures.', color='lightblue')

# Narrative Log


In [ ]:
narrative_adapter = sad.NarrativelogAdapter(server_url=server)
narrative_url = narrative_adapter.source_url
try:
    # date like '2000-01-02 12:00:00'
    # str(datetime(2000, 1, 2, 12, 0, 0))
    min_date = str(datetime.strptime(min_day_obs,'%Y%m%d'))
    max_date = str(datetime.strptime(max_day_obs,'%Y%m%d'))
    print(f'Get data from {narrative_url}: {min_date} to {max_date}')
    narrative_recs,url = narrative_adapter.get_messages(
        limit=limit,
        min_date_end=min_date,
        max_date_end=max_date
    )
except Exception as err:
    narrative_recs = []
    msg = f'ERROR getting records from {url}: {err=}'
    raise Exception(msg)

print(f'Retrieved {len(narrative_recs)} records.')

In [ ]:
if narrative_recs:
    table = narrative_adapter.day_table(narrative_recs, 'date_added')
    #print(tabstr)
    #mdlist(table, color="darkblue")
    mdlist(table)
else:
    md(f'No Narrative Log records found.', color='lightblue')
    md(f'Used [API Data]({narrative_url})')

In [ ]:
if not os.environ.get('EXTERNAL_INSTANCE_URL'):
    %run ./dashboard.ipynb